- just check how it works and what's the output. 

Q: can you only do one chromosome at a time?
A: yes, you can provide chrom_subset

In [3]:
from dlem.dlem_genome import dlem_notebook

cooler_file = '/Users/tina/LoopExtrusion/data/H1.mcool'
output_path = f'/Users/tina/LoopExtrusion/results/chr10_test.tsv'
resolution = 10_000
window_size = 200
stride = 50
dev_name ='cpu'

# make sure that all parameters are appropriately scaled to resolution!

df_result = dlem_notebook(cooler_file, output_path, resolution, 
                          model_name="minimal_dlem", window_size=window_size, stride=stride,
                          chrom_subset=['chr10'], perc_nan_threshold=0.3, lr=0.5, 
                          reader_name='datareader_cooler', dev_name=dev_name, do_return_result=True)

  0%|          | 0/264 [00:00<?, ?it/s]

100%|█████████▉| 263/264 [1:43:45<00:23, 23.67s/it]


In [9]:
df_result

,chrom,start,end,patch_i,max_corr,perc_nan,importance_weights,left,right
0,chr10,0,10000,0,0.793973,0.02,0.98,1.000000,0.990000
1,chr10,10000,20000,0,0.793973,0.02,1.96,0.581865,0.990000
2,chr10,20000,30000,0,0.793973,0.02,2.94,0.413492,0.990000
3,chr10,30000,40000,0,0.793973,0.02,3.92,0.327341,0.990000
4,chr10,40000,50000,0,0.793973,0.02,4.90,0.281918,0.990000
...,...,...,...,...,...,...,...,...,...
52795,chr10,133450000,133460000,263,0.772863,0.00,5.00,0.990000,0.986033
52796,chr10,133460000,133470000,263,0.772863,0.00,4.00,0.990000,1.000000
52797,chr10,133470000,133480000,263,0.772863,0.00,3.00,0.990000,0.994332
52798,chr10,133480000,133490000,263,0.772863,0.00,2.00,0.990000,1.000000


In [11]:
import pandas as pd

wide_df = pd.pivot_table(
    df_result, 
    index=['chrom', 'start', 'end'],
    columns='patch_i',
    values=['max_corr', 'perc_nan', 'importance_weights', 'left', 'right']
)

In [ ]:
# First, create a location ID
df_result['location_id'] = df_result.groupby(['chrom', 'start', 'end']).ngroup()
# Create an instance_id for each repetition within a locatio


In [20]:
start_values = df_result['start'].unique()
instance_mapping = {}

for start_val in start_values:
    # For each start value, assign instance IDs 1-4 to the rows with that start value
    rows_with_start = df_result[df_result['start'] == start_val].index
    for i, idx in enumerate(rows_with_start):
        instance_mapping[idx] = i + 1

# Apply the mapping to create instance_id
df_result['instance_id'] = df_result.index.map(instance_mapping)


In [22]:
df_result['instance_id'].unique()

array([1, 2, 3, 4])

In [40]:
# First, create a location identifier
df_result['location_id'] = df_result['start'].map({val: i for i, val in enumerate(start_values)})

# Now pivot
wide_df = pd.pivot(
    index='location_id',
    columns='instance_id',
    values=['start', 'end', 'patch_i', 'max_corr', 'perc_nan', 
            'importance_weights', 'left', 'right'],
    data=df_result
)

# Flatten the column names
wide_df.columns = [f'{col[0]}_{col[1]}' for col in wide_df.columns]

# Reset index
wide_df = wide_df.reset_index(drop=True)

print("\nWide format dataframe:")
print(wide_df)


Wide format dataframe:
           start_1  start_2  start_3  start_4        end_1  end_2  end_3  \
0              0.0      NaN      NaN      NaN      10000.0    NaN    NaN   
1          10000.0      NaN      NaN      NaN      20000.0    NaN    NaN   
2          20000.0      NaN      NaN      NaN      30000.0    NaN    NaN   
3          30000.0      NaN      NaN      NaN      40000.0    NaN    NaN   
4          40000.0      NaN      NaN      NaN      50000.0    NaN    NaN   
...            ...      ...      ...      ...          ...    ...    ...   
13345  133450000.0      NaN      NaN      NaN  133460000.0    NaN    NaN   
13346  133460000.0      NaN      NaN      NaN  133470000.0    NaN    NaN   
13347  133470000.0      NaN      NaN      NaN  133480000.0    NaN    NaN   
13348  133480000.0      NaN      NaN      NaN  133490000.0    NaN    NaN   
13349  133490000.0      NaN      NaN      NaN  133500000.0    NaN    NaN   

       end_4  patch_i_1  patch_i_2  ...  importance_weights_3  

In [35]:
import numpy as np
def reshape_to_wide(df):
    # Identify unique start/end combinations
    unique_locs = df[['start', 'end']].drop_duplicates().reset_index(drop=True)
    
    # Create the base dataframe with constant columns
    wide_df = unique_locs.copy()
    
    # Add chrom (assuming it's constant within each start/end group)
    wide_df['chrom'] = [df[(df['start'] == s) & (df['end'] == e)]['chrom'].iloc[0] 
                        for s, e in zip(unique_locs['start'], unique_locs['end'])]
    
    # Define the columns that should have multiple instances
    varying_columns = ['patch_i', 'max_corr', 'perc_nan', 'importance_weights', 'left', 'right']
    
    # For each unique start/end combination
    for i, (s, e) in enumerate(zip(unique_locs['start'], unique_locs['end'])):
        # Get all rows with this start/end
        rows = df[(df['start'] == s) & (df['end'] == e)]
        
        # For each varying column
        for col in varying_columns:
            # Get the values for this column
            values = rows[col].values
            
            # Add a column for each instance (up to 4)
            for j in range(min(4, len(values))):
                col_name = f'{col}_{j+1}'
                if col_name not in wide_df.columns:
                    wide_df[col_name] = np.nan
                wide_df.loc[i, col_name] = values[j]
    
    return wide_df

In [36]:
wide_df_2 = reshape_to_wide(df_result)

In [37]:
wide_df_2

,start,end,chrom,patch_i_1,max_corr_1,perc_nan_1,importance_weights_1,left_1,right_1,patch_i_2,...,perc_nan_3,importance_weights_3,left_3,right_3,patch_i_4,max_corr_4,perc_nan_4,importance_weights_4,left_4,right_4
0,0,10000,chr10,0.0,0.793973,0.02,0.98,1.000000,0.990000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000,20000,chr10,0.0,0.793973,0.02,1.96,0.581865,0.990000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20000,30000,chr10,0.0,0.793973,0.02,2.94,0.413492,0.990000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30000,40000,chr10,0.0,0.793973,0.02,3.92,0.327341,0.990000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40000,50000,chr10,0.0,0.793973,0.02,4.90,0.281918,0.990000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13345,133450000,133460000,chr10,263.0,0.772863,0.00,5.00,0.990000,0.986033,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13346,133460000,133470000,chr10,263.0,0.772863,0.00,4.00,0.990000,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13347,133470000,133480000,chr10,263.0,0.772863,0.00,3.00,0.990000,0.994332,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13348,133480000,133490000,chr10,263.0,0.772863,0.00,2.00,0.990000,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
import pandas as pd

# Assuming df_result is your dataframe with instance_id already created

# Create a location identifier 
df_result['location_id'] = df_result.groupby(['start', 'end']).ngroup()

# First pivot only the columns that vary
varying_columns = ['patch_i', 'max_corr', 'perc_nan', 'importance_weights', 'left', 'right']

# Pivot to create the wide format for varying columns
wide_df = pd.pivot(
    index='location_id',
    columns='instance_id',
    values=varying_columns,
    data=df_result
)

# Flatten the column names
wide_df.columns = [f'{col[0]}_{col[1]}' for col in wide_df.columns]

# Now add the constant columns (start, end, chrom)
# Get one row for each unique location_id 
constant_df = df_result.drop_duplicates('location_id')[['location_id', 'start', 'end', 'chrom']]

# Join the constant columns with the wide format
final_df = pd.merge(constant_df, wide_df, on='location_id')

# Drop the location_id if not needed
final_df = final_df.drop(columns=['location_id'])

# Reset index
final_df = final_df.reset_index(drop=True)

print("\nWide format dataframe with single values for start and end:")
print(final_df)


Wide format dataframe with single values for start and end:
           start        end  chrom  patch_i_1  patch_i_2  patch_i_3  \
0              0      10000  chr10        0.0        NaN        NaN   
1          10000      20000  chr10        0.0        NaN        NaN   
2          20000      30000  chr10        0.0        NaN        NaN   
3          30000      40000  chr10        0.0        NaN        NaN   
4          40000      50000  chr10        0.0        NaN        NaN   
...          ...        ...    ...        ...        ...        ...   
13345  133450000  133460000  chr10      263.0        NaN        NaN   
13346  133460000  133470000  chr10      263.0        NaN        NaN   
13347  133470000  133480000  chr10      263.0        NaN        NaN   
13348  133480000  133490000  chr10      263.0        NaN        NaN   
13349  133490000  133500000  chr10      263.0        NaN        NaN   

       patch_i_4  max_corr_1  max_corr_2  max_corr_3  ...  \
0            NaN    0.793

In [44]:
importance_weight_cols = [col for col in final_df.columns if 'importance_weights' in col]
final_df[importance_weight_cols] = final_df[importance_weight_cols].fillna(0)

In [ ]:
importance_weight_cols = [col for col in final_df.columns if 'importance_weights_' in col]

final_df['sum_importance_weights'] = final_df[importance_weight_cols].sum(axis=1)

In [66]:
importance_weight_cols = [col for col in final_df.columns if 'importance_weights_' in col]

final_df['sum_importance_weights'] = final_df[importance_weight_cols].sum(axis=1)

# Identify left, right, and importance_weights columns
left_cols = [col for col in final_df.columns if col.startswith('left_')]
right_cols = [col for col in final_df.columns if col.startswith('right_')]
importance_weight_cols = [col for col in final_df.columns if 'importance_weights' in col]

# Create weighted columns for left
for i in range(1, 5):
    left_col = f'left_{i}'
    weight_col = f'importance_weights_{i}'
    weighted_col = f'weighted_left_{i}'
    
    # Handle NaN values - if left is NaN, the product should be 0
    # We can use numpy's where function to handle this
    import numpy as np
    
    # Create the weighted column
    final_df[weighted_col] = np.where(
        final_df[left_col].isna(),  # Condition: if left is NaN
        0,                          # Then: set to 0
        final_df[left_col] * final_df[weight_col]  # Else: multiply left by weight
    )

# Create weighted columns for right (same approach)
for i in range(1, 5):
    right_col = f'right_{i}'
    weight_col = f'importance_weights_{i}'
    weighted_col = f'weighted_right_{i}'
    
    # Handle NaN values
    final_df[weighted_col] = np.where(
        final_df[right_col].isna(),  # Condition: if right is NaN
        0,                           # Then: set to 0
        final_df[right_col] * final_df[weight_col]  # Else: multiply right by weight
    )

# Display a sample of the results
print("\nDataframe with weighted left and right columns:")
print(final_df[[
    'left_1', 'importance_weights_1', 'weighted_left_1',
    'right_1', 'weighted_right_1'
]].head())


Dataframe with weighted left and right columns:
     left_1  importance_weights_1  weighted_left_1  right_1  weighted_right_1
0  1.000000                  0.98         0.980000     0.99            0.9702
1  0.581865                  1.96         1.140454     0.99            1.9404
2  0.413492                  2.94         1.215667     0.99            2.9106
3  0.327341                  3.92         1.283177     0.99            3.8808
4  0.281918                  4.90         1.381399     0.99            4.8510


In [67]:
# First calculate the sum of all weighted left values
weighted_left_cols = [col for col in final_df.columns if col.startswith('weighted_left_')]
final_df['weighted_left_sum'] = final_df[weighted_left_cols].sum(axis=1)

# Calculate the sum of all weighted right values
weighted_right_cols = [col for col in final_df.columns if col.startswith('weighted_right_')]
final_df['weighted_right_sum'] = final_df[weighted_right_cols].sum(axis=1)

# Get the sum of weights for division
importance_weight_cols = [col for col in final_df.columns if 'importance_weights' in col]
final_df['sum_importance_weights'] = final_df[importance_weight_cols].sum(axis=1)

# Calculate weighted average for left and right
# Handle the case where sum of weights is 0 to avoid division by zero
final_df['weighted_avg_left'] = final_df.apply(
    lambda row: row['weighted_left_sum'] / row['sum_importance_weights'] 
                if row['sum_importance_weights'] > 0 else 0, 
    axis=1
)

final_df['weighted_avg_right'] = final_df.apply(
    lambda row: row['weighted_right_sum'] / row['sum_importance_weights'] 
                if row['sum_importance_weights'] > 0 else 0, 
    axis=1
)

# Display the results
print("\nDataframe with weighted averages:")
print(final_df[[
    'sum_importance_weights', 
    'weighted_left_sum', 'weighted_avg_left',
    'weighted_right_sum', 'weighted_avg_right'
]].head())


Dataframe with weighted averages:
   sum_importance_weights  weighted_left_sum  weighted_avg_left  \
0                    1.96           1.960000           1.000000   
1                    3.92           2.280909           0.581865   
2                    5.88           2.431334           0.413492   
3                    7.84           2.566354           0.327341   
4                    9.80           2.762798           0.281918   

   weighted_right_sum  weighted_avg_right  
0              1.9404                0.99  
1              3.8808                0.99  
2              5.8212                0.99  
3              7.7616                0.99  
4              9.7020                0.99  


In [51]:
final_df.columns

Index(['start', 'end', 'chrom', 'patch_i_1', 'patch_i_2', 'patch_i_3',
       'patch_i_4', 'max_corr_1', 'max_corr_2', 'max_corr_3', 'max_corr_4',
       'perc_nan_1', 'perc_nan_2', 'perc_nan_3', 'perc_nan_4',
       'importance_weights_1', 'importance_weights_2', 'importance_weights_3',
       'importance_weights_4', 'left_1', 'left_2', 'left_3', 'left_4',
       'right_1', 'right_2', 'right_3', 'right_4', 'sum_importance_weights',
       'weighted_left_1', 'weighted_left_2', 'weighted_left_3',
       'weighted_left_4', 'weighted_right_1', 'weighted_right_2',
       'weighted_right_3', 'weighted_right_4', 'weighted_left_sum',
       'weighted_right_sum', 'weighted_avg_left', 'weighted_avg_right'],
      dtype='object')

In [68]:
final_df[['right_1', 'right_2', 'right_3', 'right_4', 'importance_weights_1', 'importance_weights_2', 'importance_weights_3',
       'importance_weights_4', 'sum_importance_weights', 'weighted_right_1', 'weighted_right_2',
       'weighted_right_3', 'weighted_right_4', 'weighted_right_sum', 'weighted_avg_right']]

,right_1,right_2,right_3,right_4,importance_weights_1,importance_weights_2,importance_weights_3,importance_weights_4,sum_importance_weights,weighted_right_1,weighted_right_2,weighted_right_3,weighted_right_4,weighted_right_sum,weighted_avg_right
0,0.990000,NaN,NaN,NaN,0.98,0.0,0.0,0.0,1.96,0.970200,0.0,0.0,0.0,1.940400,0.990000
1,0.990000,NaN,NaN,NaN,1.96,0.0,0.0,0.0,3.92,1.940400,0.0,0.0,0.0,3.880800,0.990000
2,0.990000,NaN,NaN,NaN,2.94,0.0,0.0,0.0,5.88,2.910600,0.0,0.0,0.0,5.821200,0.990000
3,0.990000,NaN,NaN,NaN,3.92,0.0,0.0,0.0,7.84,3.880800,0.0,0.0,0.0,7.761600,0.990000
4,0.990000,NaN,NaN,NaN,4.90,0.0,0.0,0.0,9.80,4.851000,0.0,0.0,0.0,9.702000,0.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13345,0.986033,NaN,NaN,NaN,5.00,0.0,0.0,0.0,10.00,4.930166,0.0,0.0,0.0,9.860332,0.986033
13346,1.000000,NaN,NaN,NaN,4.00,0.0,0.0,0.0,8.00,4.000000,0.0,0.0,0.0,8.000000,1.000000
13347,0.994332,NaN,NaN,NaN,3.00,0.0,0.0,0.0,6.00,2.982996,0.0,0.0,0.0,5.965992,0.994332
13348,1.000000,NaN,NaN,NaN,2.00,0.0,0.0,0.0,4.00,2.000000,0.0,0.0,0.0,4.000000,1.000000


In [53]:
final_df['sum_importance_weights'] = final_df[importance_weight_cols].sum(axis=1)

In [69]:
final_df

,start,end,chrom,patch_i_1,patch_i_2,patch_i_3,patch_i_4,max_corr_1,max_corr_2,max_corr_3,...,weighted_left_3,weighted_left_4,weighted_right_1,weighted_right_2,weighted_right_3,weighted_right_4,weighted_left_sum,weighted_right_sum,weighted_avg_left,weighted_avg_right
0,0,10000,chr10,0.0,NaN,NaN,NaN,0.793973,NaN,NaN,...,0.0,0.0,0.970200,0.0,0.0,0.0,1.960000,1.940400,1.000000,0.990000
1,10000,20000,chr10,0.0,NaN,NaN,NaN,0.793973,NaN,NaN,...,0.0,0.0,1.940400,0.0,0.0,0.0,2.280909,3.880800,0.581865,0.990000
2,20000,30000,chr10,0.0,NaN,NaN,NaN,0.793973,NaN,NaN,...,0.0,0.0,2.910600,0.0,0.0,0.0,2.431334,5.821200,0.413492,0.990000
3,30000,40000,chr10,0.0,NaN,NaN,NaN,0.793973,NaN,NaN,...,0.0,0.0,3.880800,0.0,0.0,0.0,2.566354,7.761600,0.327341,0.990000
4,40000,50000,chr10,0.0,NaN,NaN,NaN,0.793973,NaN,NaN,...,0.0,0.0,4.851000,0.0,0.0,0.0,2.762798,9.702000,0.281918,0.990000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13345,133450000,133460000,chr10,263.0,NaN,NaN,NaN,0.772863,NaN,NaN,...,0.0,0.0,4.930166,0.0,0.0,0.0,9.900000,9.860332,0.990000,0.986033
13346,133460000,133470000,chr10,263.0,NaN,NaN,NaN,0.772863,NaN,NaN,...,0.0,0.0,4.000000,0.0,0.0,0.0,7.920000,8.000000,0.990000,1.000000
13347,133470000,133480000,chr10,263.0,NaN,NaN,NaN,0.772863,NaN,NaN,...,0.0,0.0,2.982996,0.0,0.0,0.0,5.940000,5.965992,0.990000,0.994332
13348,133480000,133490000,chr10,263.0,NaN,NaN,NaN,0.772863,NaN,NaN,...,0.0,0.0,2.000000,0.0,0.0,0.0,3.960000,4.000000,0.990000,1.000000


In [63]:
final_df[['importance_weights_1', 'importance_weights_2', 'importance_weights_3',
       'importance_weights_4', 'sum_importance_weights']]

,importance_weights_1,importance_weights_2,importance_weights_3,importance_weights_4,sum_importance_weights
0,0.98,0.0,0.0,0.0,0.98
1,1.96,0.0,0.0,0.0,1.96
2,2.94,0.0,0.0,0.0,2.94
3,3.92,0.0,0.0,0.0,3.92
4,4.90,0.0,0.0,0.0,4.90
...,...,...,...,...,...
13345,5.00,0.0,0.0,0.0,5.00
13346,4.00,0.0,0.0,0.0,4.00
13347,3.00,0.0,0.0,0.0,3.00
13348,2.00,0.0,0.0,0.0,2.00


In [62]:
importance_weight_cols = [col for col in final_df.columns if 'importance_weights_' in col]

final_df['sum_importance_weights'] = final_df[importance_weight_cols].sum(axis=1)